## Dependencies

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import Adam
import tf2onnx

## Loading Data

In [9]:
# Load features into a DataFrame
features_df = pd.read_csv("SaliencyLog.csv")

In [10]:
# Features Head
print(features_df.head())

   Frame  ObjectID   PosX   PosY    PosZ  Motion  AngularVelocity  Proximity  \
0      1        11  12.47  11.09  -13.35  0.0000              0.0     0.3585   
1      1         8  24.87  41.89  -65.80  0.0000              0.0     0.5811   
2      1         1 -85.66  68.10   98.33  0.9996              0.0     0.9893   
3      1         6   9.50   0.54  114.49  0.5034              0.0     0.2668   
4      1         4  48.11   0.60  114.20  0.9991              0.0     0.3413   

    Color  Luminance  IsBest  
0  0.1716     0.0043       0  
1  0.1133     0.0927       0  
2  0.6870     0.7089       0  
3  0.6870     0.7089       0  
4  0.2227     0.1050       0  


In [11]:
# Features info
features_df.describe()

,Frame,ObjectID,PosX,PosY,PosZ,Motion,AngularVelocity,Proximity,Color,Luminance,IsBest
count,2379.000000,2379.000000,2379.000000,2379.000000,2379.000000,2379.000000,2379.000000,2379.000000,2379.000000,2379.000000,2379.000000
mean,92.000000,6.000000,27.049172,22.682278,-9.094758,0.312094,0.015990,0.492446,0.355409,0.282466,0.076923
std,52.837867,3.742444,42.357332,23.447462,90.523311,0.297348,0.059494,0.319735,0.226760,0.289883,0.266525
min,1.000000,0.000000,-85.660000,0.170000,-241.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,46.000000,3.000000,6.050000,0.640000,-46.070000,0.000000,0.000000,0.155900,0.222700,0.092700,0.000000
50%,92.000000,6.000000,24.360000,12.310000,-13.350000,0.487600,0.002500,0.408300,0.234600,0.105000,0.000000
75%,138.000000,9.000000,57.915000,35.500000,40.210000,0.596400,0.012600,0.715900,0.687000,0.708900,0.000000
max,183.000000,12.000000,129.660000,68.100000,212.710000,0.999600,0.971000,0.995100,0.687000,0.708900,1.000000


In [12]:
# Finding nulls
features_df.isna().sum()

Frame              0
ObjectID           0
PosX               0
PosY               0
PosZ               0
Motion             0
AngularVelocity    0
Proximity          0
Color              0
Luminance          0
IsBest             0
dtype: int64

In [13]:
# Getting data types of cols
features_df.dtypes

Frame                int64
ObjectID             int64
PosX               float64
PosY               float64
PosZ               float64
Motion             float64
AngularVelocity    float64
Proximity          float64
Color              float64
Luminance          float64
IsBest               int64
dtype: object

In [14]:
# Load targets into a DataFrame
targets_df = pd.read_csv("SaliencyLogOutput.csv")

In [15]:
# Targets Head
print(targets_df.tail())

     Frame  BodyForwardX  BodyForwardY  BodyForwardZ  HeadForwardX  \
178    179          0.71           0.0         -0.71          0.97   
179    180          0.71           0.0         -0.71          0.97   
180    181          0.71           0.0         -0.71          0.97   
181    182          0.71           0.0         -0.71          0.97   
182    183          0.71           0.0         -0.71          0.96   

     HeadForwardY  HeadForwardZ  
178         -0.02         -0.23  
179         -0.02         -0.24  
180         -0.02         -0.25  
181         -0.02         -0.26  
182         -0.02         -0.27  


In [16]:
# Targets Info
targets_df.describe()

,Frame,BodyForwardX,BodyForwardY,BodyForwardZ,HeadForwardX,HeadForwardY,HeadForwardZ
count,183.000000,1.830000e+02,183.0,1.830000e+02,183.000000,183.000000,183.000000
mean,92.000000,7.100000e-01,0.0,-7.100000e-01,0.556721,0.115628,-0.513989
std,52.971691,1.113269e-16,0.0,1.113269e-16,0.491108,0.170292,0.377566
min,1.000000,7.100000e-01,0.0,-7.100000e-01,-0.440000,-0.260000,-1.000000
25%,46.500000,7.100000e-01,0.0,-7.100000e-01,-0.110000,-0.020000,-0.960000
50%,92.000000,7.100000e-01,0.0,-7.100000e-01,0.860000,0.130000,-0.410000
75%,137.500000,7.100000e-01,0.0,-7.100000e-01,0.980000,0.240000,-0.165000
max,183.000000,7.100000e-01,0.0,-7.100000e-01,1.000000,0.550000,0.060000


In [17]:
# Finding nulls
targets_df.isna().sum()

Frame           0
BodyForwardX    0
BodyForwardY    0
BodyForwardZ    0
HeadForwardX    0
HeadForwardY    0
HeadForwardZ    0
dtype: int64

In [18]:
# Getting data types of cols
targets_df.dtypes

Frame             int64
BodyForwardX    float64
BodyForwardY    float64
BodyForwardZ    float64
HeadForwardX    float64
HeadForwardY    float64
HeadForwardZ    float64
dtype: object

## Data Seperation

In [19]:
# Columns to cut
cols_to_move = ["BodyForwardX", "BodyForwardY", "BodyForwardZ"]
subset = targets_df[cols_to_move]

In [20]:
# Repeat each row 13 times
subset_repeated = subset.loc[np.repeat(subset.index, 13)].reset_index(drop=True)

In [21]:
# Paste into df2 (side by side or append depending on your need)
# --- if you want to append as new columns (must match row count in df2) ---
features_df = pd.concat([features_df.reset_index(drop=True), subset_repeated], axis=1)

In [22]:
targets_df = targets_df.drop(columns=cols_to_move)

In [23]:
# Features Head
print(features_df.tail())

      Frame  ObjectID   PosX   PosY    PosZ  Motion  AngularVelocity  \
2374    183         5  -6.35   0.64  -56.07  0.5747           0.0014   
2375    183         0   6.00   0.60  107.32  0.5317           0.0030   
2376    183         2  57.92  17.00  -60.35  0.6501           0.0194   
2377    183        10 -14.35  12.31  -33.39  0.0000           0.0000   
2378    183         9  35.60   8.55   11.97  0.0000           0.0000   

      Proximity   Color  Luminance  IsBest  BodyForwardX  BodyForwardY  \
2374     0.1278  0.2227     0.1050       0          0.71           0.0   
2375     0.0944  0.2227     0.1050       0          0.71           0.0   
2376     0.3991  0.1221     0.0338       0          0.71           0.0   
2377     0.3073  0.6870     0.7089       0          0.71           0.0   
2378     0.1559  0.3187     0.0203       0          0.71           0.0   

      BodyForwardZ  
2374         -0.71  
2375         -0.71  
2376         -0.71  
2377         -0.71  
2378         -0.7

In [24]:
# Targets Head
print(targets_df.tail())

     Frame  HeadForwardX  HeadForwardY  HeadForwardZ
178    179          0.97         -0.02         -0.23
179    180          0.97         -0.02         -0.24
180    181          0.97         -0.02         -0.25
181    182          0.97         -0.02         -0.26
182    183          0.96         -0.02         -0.27


In [25]:
# Split targets and features
features = ["PosX", "PosY", "PosZ", "Motion", "AngularVelocity", "Proximity", "Color", "Luminance", "BodyForwardX", "BodyForwardY", "BodyForwardZ"]
targets = ["IsBest", "HeadForwardX", "HeadForwardY", "HeadForwardZ"]

In [26]:
pivot_data = []
for frame, group in features_df.groupby("Frame"):
    row = {"Frame": frame}
    for _, obj in group.iterrows():
        for feat in features:
            row[f"Obj{obj['ObjectID']}_{feat}"] = obj[feat]
    
    # Check if IsBest==1 exists
    best = group.loc[group["IsBest"] == 1, "ObjectID"].values
    if len(best) > 0:
        row["Label"] = best[0]
    else:
        row["Label"] = 0 
    
    pivot_data.append(row)

pivot_df = pd.DataFrame(pivot_data)

In [27]:
# Merge with regression targets
pivot_df["Frame"] = pivot_df["Frame"].astype(int)
targets_df["Frame"] = targets_df["Frame"].astype(int)

pivot_df = pivot_df.merge(targets_df, on="Frame")

## Train-Test-Split

In [28]:
# Prepare data
X = pivot_df.drop(["Frame", "Label", "HeadForwardX", "HeadForwardY", "HeadForwardZ"], axis=1)
y_class = pivot_df["Label"]
y_reg = pivot_df[["HeadForwardX", "HeadForwardY", "HeadForwardZ"]]

In [29]:
# Printing head
X.head()

,Obj11.0_PosX,Obj11.0_PosY,Obj11.0_PosZ,Obj11.0_Motion,Obj11.0_AngularVelocity,Obj11.0_Proximity,Obj11.0_Color,Obj11.0_Luminance,Obj11.0_BodyForwardX,Obj11.0_BodyForwardY,...,Obj9.0_PosY,Obj9.0_PosZ,Obj9.0_Motion,Obj9.0_AngularVelocity,Obj9.0_Proximity,Obj9.0_Color,Obj9.0_Luminance,Obj9.0_BodyForwardX,Obj9.0_BodyForwardY,Obj9.0_BodyForwardZ
0,12.47,11.09,-13.35,0.0,0.0,0.3585,0.1716,0.0043,0.71,0.0,...,8.55,11.97,0.0,0.0,0.1559,0.3187,0.0203,0.71,0.0,-0.71
1,12.47,11.09,-13.35,0.0,0.0,0.3585,0.1716,0.0043,0.71,0.0,...,8.55,11.97,0.0,0.0,0.1559,0.3187,0.0203,0.71,0.0,-0.71
2,12.47,11.09,-13.35,0.0,0.0,0.3585,0.1716,0.0043,0.71,0.0,...,8.55,11.97,0.0,0.0,0.1559,0.3187,0.0203,0.71,0.0,-0.71
3,12.47,11.09,-13.35,0.0,0.0,0.3585,0.1716,0.0043,0.71,0.0,...,8.55,11.97,0.0,0.0,0.1559,0.3187,0.0203,0.71,0.0,-0.71
4,12.47,11.09,-13.35,0.0,0.0,0.3585,0.1716,0.0043,0.71,0.0,...,8.55,11.97,0.0,0.0,0.1559,0.3187,0.0203,0.71,0.0,-0.71


In [30]:
# Train/test split
X_train, X_test, y_class_train, y_class_test, y_reg_train, y_reg_test = train_test_split(
    X, y_class, y_reg, test_size=0.2, random_state=42
)

## CLASSIFICATION MODEL

## Model Training

In [31]:
# Classification model
clf_model = models.Sequential([
    layers.Input(shape=(X.shape[1],)),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(64, activation="relu"),
    layers.Dense(13, activation="softmax")  # softmax for classification
])

In [32]:
# Create Adam optimizer with custom learning rate
adam_optimizer = Adam(learning_rate=0.001)  # 0.001 is default

# Compile the model
clf_model.compile(optimizer=adam_optimizer,
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

In [33]:
# Model Parameters
clf_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │          18,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 13)                  │             845 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 27,533 (107.55 KB)

 Trainable params: 27,533 (107.55 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
# Convert to numpy, enforce int32, flatten to 1D
y_class_train = y_class_train.to_numpy().astype("int32").reshape(-1,)
y_class_test  = y_class_test.to_numpy().astype("int32").reshape(-1,)

In [35]:
print(y_class_train.shape, y_class_train.dtype)
print(y_class_test.shape, y_class_test.dtype)

(146,) int32
(37,) int32


In [36]:
# Train the network for classification task
clf_model.fit(X_train, y_class_train, epochs=75, batch_size=32, validation_data=(X_test, y_class_test))

Epoch 1/75
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.3268 - loss: 27.7533 - val_accuracy: 0.8378 - val_loss: 2.0476
Epoch 2/75
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4795 - loss: 11.3233 - val_accuracy: 0.7027 - val_loss: 3.1671
Epoch 3/75
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6609 - loss: 6.0979 - val_accuracy: 0.8108 - val_loss: 2.3330
Epoch 4/75
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7518 - loss: 5.5582 - val_accuracy: 0.8378 - val_loss: 1.5187
Epoch 5/75
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7927 - loss: 3.8231 - val_accuracy: 0.8919 - val_loss: 1.2023
Epoch 6/75
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6331 - loss: 5.9779 - val_accuracy: 0.8649 - val_loss: 0.8877
Epoch 7/75
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7299 - loss: 4.7534 - val_accuracy: 0.8919 - val_loss: 0.4417
Epoch 8/75
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7471 - loss: 4.4388 - val_accuracy: 0.9189 - val_loss: 0.677

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8659 - loss: 0.3901 - val_accuracy: 0.9189 - val_loss: 0.2056
Epoch 53/75
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8971 - loss: 0.4377 - val_accuracy: 0.9459 - val_loss: 0.1630
Epoch 54/75
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8760 - loss: 0.4792 - val_accuracy: 0.9459 - val_loss: 0.1044
Epoch 55/75
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8915 - loss: 0.4266 - val_accuracy: 0.9730 - val_loss: 0.0838
Epoch 56/75
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8846 - loss: 0.5030 - val_accuracy: 0.9459 - val_loss: 0.1086
Epoch 57/75
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8736 - loss: 0.4795 - val_accuracy: 0.9459 - val_loss: 0.1523
Epoch 58/75
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8747 - loss: 0.3580 - val_accuracy: 0.9189 - val_loss: 0.1761
Epoch 59/75
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8772 - loss: 0.4492 - val_accuracy: 0.9189 - val_loss: 0.2081
Epoc

## Model Evaluation

In [37]:
# Evaluate classification
loss, acc = clf_model.evaluate(X_test, y_class_test, verbose=0)
print(f"Classification - Loss: {loss:.4f}, Accuracy: {acc:.4f}")

Classification - Loss: 0.1794, Accuracy: 0.9189


## Save Architecture and Weights

In [38]:
# Save architecture
model_json = clf_model.to_json()
with open("clf_model.json", "w") as json_file:
    json_file.write(model_json)

# Save weights
clf_model.save_weights("clf_model.weights.h5")
print("Model saved to clf_model.json + clf_model.weights.h5")

Model saved to clf_model.json + clf_model.weights.h5


## REGRESSION MODEL

In [39]:
# Regression model
reg_model = models.Sequential([
    layers.Input(shape=(X.shape[1],)),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(64, activation="relu"),
    layers.Dense(y_reg.shape[1])  # no activation → linear output
])

In [40]:
# Create Adam optimizer with custom learning rate
adam_optimizer = Adam(learning_rate=0.001)  # 0.001 is default

# Compile the model
reg_model.compile(optimizer="adam",
                  loss="mse",
                  metrics=["mae"])

In [41]:
# Get the parameters
reg_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 128)                 │          18,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,883 (105.01 KB)

 Trainable params: 26,883 (105.01 KB)

 Non-trainable params: 0 (0.00 B)

In [42]:
# Train the model
reg_model.fit(X_train, y_reg_train, epochs=150, batch_size=16, validation_data=(X_test, y_reg_test))

Epoch 1/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 521.7086 - mae: 17.4475 - val_loss: 117.6959 - val_mae: 7.8486
Epoch 2/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 158.2751 - mae: 9.8690 - val_loss: 25.9747 - val_mae: 4.0422
Epoch 3/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 79.5189 - mae: 7.3215 - val_loss: 8.2604 - val_mae: 2.3419
Epoch 4/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 75.4191 - mae: 6.8189 - val_loss: 5.7707 - val_mae: 1.8268
Epoch 5/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 38.5087 - mae: 4.9079 - val_loss: 15.0572 - val_mae: 3.1982
Epoch 6/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 37.5009 - mae: 4.8499 - val_loss: 4.0776 - val_mae: 1.5687
Epoch 7/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 27.9953 - mae: 4.2399 - val_loss: 5.0805 - val_mae: 1.6790
Epoch 8/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 25.6776 - mae: 3.9962 - val_loss: 5.3556 - val_mae: 1.8656
Epoch 9/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.8086 - mae: 0.6885 - val_loss: 0.2090 - val_mae: 0.3309
Epoch 55/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.7533 - mae: 0.6413 - val_loss: 0.1958 - val_mae: 0.3214
Epoch 56/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.5854 - mae: 0.5587 - val_loss: 0.1916 - val_mae: 0.3207
Epoch 57/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.7440 - mae: 0.6594 - val_loss: 0.1812 - val_mae: 0.3150
Epoch 58/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.8410 - mae: 0.6652 - val_loss: 0.1794 - val_mae: 0.3170
Epoch 59/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.6891 - mae: 0.6111 - val_loss: 0.1860 - val_mae: 0.3268
Epoch 60/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.6791 - mae: 0.5773 - val_loss: 0.1755 - val_mae: 0.3192
Epoch 61/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.6088 - mae: 0.5588 - val_loss: 0.1788 - val_mae: 0.3219
Epoch 62/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.5292 

Epoch 108/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2746 - mae: 0.3952 - val_loss: 0.1397 - val_mae: 0.3108
Epoch 109/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2438 - mae: 0.3580 - val_loss: 0.1390 - val_mae: 0.3104
Epoch 110/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2185 - mae: 0.3615 - val_loss: 0.1375 - val_mae: 0.3087
Epoch 111/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2129 - mae: 0.3519 - val_loss: 0.1358 - val_mae: 0.3070
Epoch 112/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2599 - mae: 0.3681 - val_loss: 0.1341 - val_mae: 0.3050
Epoch 113/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2034 - mae: 0.3551 - val_loss: 0.1327 - val_mae: 0.3038
Epoch 114/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2589 - mae: 0.3706 - val_loss: 0.1307 - val_mae: 0.3015
Epoch 115/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2168 - mae: 0.3634 - val_loss: 0.1311 - val_mae: 0.3020
Epoch 116/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9m

## Model Evaluation

In [43]:
# Evaluate regression
loss, mse = reg_model.evaluate(X_test, y_reg_test, verbose=0)
print(f"Regression - Loss: {loss:.4f}, MSE: {mse:.4f}")

Regression - Loss: 0.0796, MSE: 0.2426


## Save Architecture and Weights

In [44]:
# Save architecture
model_json = reg_model.to_json()
with open("reg_model.json", "w") as json_file:
    json_file.write(model_json)

# Save weights
clf_model.save_weights("reg_model.weights.h5")
print("Model saved to reg_model.json + reg_model.weights.h5")

Model saved to reg_model.json + reg_model.weights.h5


## Loading Module

In [45]:
# If you want to load any model architecture and use it
# Or load weights and skip training
# Convert the following comments into code and run

# Un-comment & Enter the model name below (clf_model or reg_model)
# ~~~~~~~~~~~~~~~~~~~~~~~~~
# model_name = ?
# ~~~~~~~~~~~~~~~~~~~~~~~~~

# To Load architecture
# Un-comment below block 
# ~~~~~~~~~~~~~~~~~~~~~~~~~
# with open("model_name.json", "r") as json_file:
#     loaded_model_json = json_file.read()

# loaded_model = model_from_json(loaded_model_json)
# ~~~~~~~~~~~~~~~~~~~~~~~~~

# To Load weights
# Un-comment below block 
# ~~~~~~~~~~~~~~~~~~~~~~~~~
# loaded_model.load_weights("model_name.weights.h5")
# print("Model loaded successfully")
# ~~~~~~~~~~~~~~~~~~~~~~~~~

## Saving to ONNX

In [59]:
# For classification model
clf_model.export("saved_clf_model")  # this is the new way

INFO:tensorflow:Assets written to: saved_clf_model\assets


INFO:tensorflow:Assets written to: saved_clf_model\assets


Saved artifact at 'saved_clf_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 143), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 13), dtype=tf.float32, name=None)
Captures:
  2601531920784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2601531921936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2601531923280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2601531921168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2601531922320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2601531923856: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
# Go to your directory where the saved model is present
# And run the following command in command line interface to create an ONNX file (Un-Comment & Copy the line below)
# python -m tf2onnx.convert --saved-model saved_clf_model --output clf_model.onnx --opset 13

In [62]:
# For classification model
reg_model.export("saved_reg_model")  # this is the new way

INFO:tensorflow:Assets written to: saved_reg_model\assets


INFO:tensorflow:Assets written to: saved_reg_model\assets


Saved artifact at 'saved_reg_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 143), dtype=tf.float32, name='keras_tensor_5')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  2601531926160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2601531931152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2601531929040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2601531930768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2601531926736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2601531931536: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
# Go to your directory where the saved model is present
# And run the following command in command line interface to create an ONNX file (Un-Comment & Copy the line below)
# python -m tf2onnx.convert --saved-model saved_reg_model --output reg_model.onnx --opset 13